In [149]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path

from Scripts.pywin32_testall import project_root
from scipy.stats import obrientransform

In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

In [4]:
Root_dir = Path.cwd().parent

In [5]:
data_dir = Path(Root_dir.joinpath('data/raw/tlc/yellow/2023'))
zone_path = Path(Root_dir.joinpath('data/raw/tlc/misc/taxi_zone_lookup.csv'))

In [6]:
file_1 = Path(data_dir.joinpath('yellow_tripdata_2023-01.parquet'))

In [7]:
df = pd.read_parquet(file_1)
print(df.shape)
df.head()

(3066766, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [8]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [9]:
size_in_bytes = df.memory_usage(deep=True,index=False).sum()
size_in_mb = size_in_bytes / (1024**2)
size_in_gb = size_in_bytes / (1024**3)
print(f'{size_in_mb:.3f} MB, {size_in_gb:.3f} GB')

588.462 MB, 0.575 GB


In [10]:
df.memory_usage(deep=True,index=False).groupby(df.dtypes.astype(str)).sum().sort_values(ascending=True)

datetime64[us]     49068256
int64              98136512
object            175433166
float64           294409536
dtype: int64

In [11]:
Top_10 = df.memory_usage(deep=True,index=False).sort_values(ascending=False).head(10).to_dict()

In [12]:
Top_10

{'store_and_fwd_flag': 175433166,
 'tpep_pickup_datetime': 24534128,
 'VendorID': 24534128,
 'tpep_dropoff_datetime': 24534128,
 'passenger_count': 24534128,
 'trip_distance': 24534128,
 'RatecodeID': 24534128,
 'PULocationID': 24534128,
 'DOLocationID': 24534128,
 'payment_type': 24534128}

In [13]:
memory_percentage_column = {}
for key, value in Top_10.items():
    memory_percentage_column[key] = (value / size_in_bytes * 100)

In [14]:
avg_memory_row = {}
for key, value in df.memory_usage(deep=True,index=False).sort_values(ascending=False).to_dict().items():
    avg_memory_row[key] = value/df.shape[0]

In [15]:
avg_memory_row

{'store_and_fwd_flag': 57.20461424184304,
 'tpep_pickup_datetime': 8.0,
 'VendorID': 8.0,
 'tpep_dropoff_datetime': 8.0,
 'passenger_count': 8.0,
 'trip_distance': 8.0,
 'RatecodeID': 8.0,
 'PULocationID': 8.0,
 'DOLocationID': 8.0,
 'payment_type': 8.0,
 'fare_amount': 8.0,
 'extra': 8.0,
 'mta_tax': 8.0,
 'tip_amount': 8.0,
 'tolls_amount': 8.0,
 'improvement_surcharge': 8.0,
 'total_amount': 8.0,
 'congestion_surcharge': 8.0,
 'airport_fee': 8.0}

In [16]:
row_na = df.isna().any(axis=1)
frac_row_na = row_na.mean()

In [86]:
percentage_na = round(frac_row_na*100, 3)
percentage_na

np.float64(2.339)

In [18]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [154]:
duration_min = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds()/60
absurd_timings = (duration_min <= 0).sum()
absurd_timings

np.int64(1121)

In [92]:
pct_absurd_timings = round((absurd_timings/df.shape[0])*100,3)
pct_absurd_timings

np.float64(0.037)

In [21]:
print(duration_min.describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).to_string())

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
1%       7.833333e-01
5%       3.300000e+00
50%      1.151667e+01
95%      3.646667e+01
99%      5.725000e+01
max      1.002918e+04


In [22]:
print(df['trip_distance'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).to_string())

count    3.066766e+06
mean     3.847342e+00
std      2.495838e+02
min      0.000000e+00
1%       0.000000e+00
5%       5.000000e-01
50%      1.800000e+00
95%      1.432000e+01
99%      2.006000e+01
max      2.589281e+05


In [128]:
le_0distance = (df['trip_distance'] <= 0).sum()
pct_le_0distance = round((((df['trip_distance'] <= 0).sum())/df.shape[0])*100,3)

In [99]:
df['trip_distance'].describe(percentiles=[0.999])

{'count': 3066766.0,
 'mean': 3.8473420306603128,
 'std': 249.58375606858064,
 'min': 0.0,
 '50%': 1.8,
 '99.9%': 28.8,
 'max': 258928.15}

In [25]:
soft_cap = max(df['trip_distance'].describe(percentiles=[0.999]).to_dict()['99.9%'],50)

In [26]:
soft_cap

50

In [103]:
n_over_cap = (df['trip_distance'] > soft_cap).sum()
pct_over_cap = round ( ((df['trip_distance'] > soft_cap).sum())/df.shape[0]*100,3)

In [104]:
print('The 0.1% above cap are',n_over_cap,'the percentage is',pct_over_cap)

The 0.1% above cap are 387 the percentage is 0.013


In [29]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [30]:
fare_n_charges = ['fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge','airport_fee']

In [42]:
comp_sum = df[fare_n_charges].fillna(0).sum(axis=1)
total_sum = df['total_amount']

In [132]:
difference_of_amount = abs(comp_sum - total_sum)
fare_missmatch = (total_sum != comp_sum).sum()
pct_fare_missmatch = round(fare_missmatch/df.shape[0]*100,3)
n_total_nonpos = ((total_sum - comp_sum) < 0).sum()

In [58]:
positive_amount = total_sum > 0
rel_diff = pd.Series(0.0, index = df.index)
rel_diff[positive_amount] = difference_of_amount[positive_amount]/total_sum[positive_amount]

In [62]:
compared_difference = (difference_of_amount > 2) | (positive_amount & (rel_diff > 0.2))
100 * compared_difference.sum()/df.shape[0]

np.float64(26.24239997443561)

In [66]:
~positive_amount.sum()

np.int64(-3040995)

In [152]:
import pandas.api
rows = []
for column in df.columns:
    s = df[column]
    dtype_str = str(s.dtype)
    missing_pct = df[column].isna().mean()*100
    unique_ct = df[column].nunique(dropna=True)
    if df[column].notna().any():
        ex = s.dropna().iloc[0]
        if pandas.api.types.is_datetime64_any_dtype(s):
            try:
                example = pd.to_datetime(ex).isoformat()
            except Exception:
                example = str(ex)
        else:
            example = str(ex)
    else:
        example = None

    rows.append(
        {
            'name' : column,
            'dtype' : dtype_str,
            'missing_%' : missing_pct,
            'unique_count' : unique_ct,
            'example' : example
        }
    )

profile = pd.DataFrame(rows,columns=['name','dtype','missing_%','unique_count','example'])
profile = profile.sort_values(by=['missing_%','name'], ascending=[False,True]).reset_index(drop=True)

In [153]:
output_path = Path(Root_dir.joinpath('reports/tables/overview_columns.csv'))
output_path.parent.mkdir(parents=True, exist_ok=True)
profile.to_csv(output_path, index=False)

In [147]:
overview_dict ={
    'rows' : df.shape[0],
    'columns' : df.shape[1],
    'memory' : round(size_in_mb,3),
    'bytes_per_row' : round(df.memory_usage(deep=True).sum()/df.shape[0],3),
    'pct_row_with_any_missing' : round(df.isna().any(axis=1).mean()*100,3),
    'Number_of_negative_duration' : absurd_timings,
    'pct_of_negative_duration' : pct_absurd_timings,
    'distance_cap_miles' : soft_cap,
    'n_over_cap' : n_over_cap,
    'pct_over_cap' : pct_over_cap,
    'n_le_0distance' : le_0distance,
    'pct_le_0distance' : pct_le_0distance,
    'fare_missmatch' : fare_missmatch,
    'pct_fare_missmatch' : pct_fare_missmatch,
    'n_total_nonpos' : n_total_nonpos
}

In [156]:
output_path = Path(Root_dir.joinpath('reports/tables/overview_stats.json'))
output_path.parent.mkdir(parents=True, exist_ok=True)
overview_dict = { k:float(v) for k,v in overview_dict.items()}
with open(output_path, 'w') as f:
    json.dump(overview_dict, f,indent=4)